In [ ]:
import tensorflow as tf
import numpy as np
import datetime
import os
import glob

In [ ]:
MAIN_AUC_FILE_DIR = '/content/drive/My Drive/OGR+/DenseNet/logs/aucperepoch.csv'

BASELINE_MODEL_CP_DIR = '/content/drive/My Drive/OGR/DenseNet/model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5'
BASELINE_MODEL_LOG_DIR ='/content/drive/My Drive/OGR/DenseNet/logs'
BASELINE_MODEL_AUC_LOG_DIR = '/content/drive/My Drive/OGR/DenseNet/logs/aucperepoch.csv'

MODEL_CP_DIR = '/content/drive/My Drive/OGR+/DenseNet/model_{type}/{type}'
MODEL_LOG_DIR = '/content/drive/My Drive/OGR+/DenseNet/logs_{type}'

MODEL_TYPES = ['full', 'frontal', 'combined', 'side']

INIT_LR = 1e-4
BS=32
WK=5
EPOCH=10
SELECTED_VIEW = 'pa_la'

AUC_BS = 512

## **Data Pipeline**

### Image Augmentation

In [ ]:
import math
from tensorflow.keras import backend as K

def get_mat(rotation,height_zoom=1,width_zoom=1):
    rotation = math.pi * rotation / 180.
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    return K.dot(rotation_matrix, zoom_matrix)

In [ ]:
def transform(image):
    DIM = 224
    XDIM = DIM%2  
    rot = 10. * tf.random.normal([1],dtype='float32')
    m = get_mat(rot) 
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
    return tf.reshape(d,[DIM,DIM,3])

In [ ]:
def augment_img(image):
  image = tf.image.resize(image, [264,264])
  image = tf.image.random_brightness(image, 0.4)
  image = tf.image.random_crop(image, [224,224,3])
  image = tf.image.random_flip_left_right(image)
  image = transform(image)
  return image

### TFrecord

In [ ]:
def augment_img_wot(image):
  image = tf.image.resize(image, [264,264])
  image = tf.image.crop_to_bounding_box(image, 20,20, 224,224)
  return image

In [ ]:
def read_tfrecord(example, output_mode='both', with_transform=False):
    features = {
        "image_front": tf.io.FixedLenFeature([], tf.string),
        "image_side": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.VarLenFeature(tf.float32),  
        "view": tf.io.FixedLenFeature([], tf.string),
        "select": tf.io.FixedLenFeature([], tf.int64),
        "study_id": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, features)
    label  = tf.reshape(tf.sparse.to_dense(example['label']), [1, 14])
    label = tf.where(tf.math.is_nan(label), tf.ones_like(label) * 0, label)
    label = tf.where(label == -1, tf.ones_like(label) * 0, label)
    label = label[0]

    select = example['select']
    if output_mode=='label':
      return label, select
    elif output_mode=='view_label':
      return label, example['view'], select
    elif output_mode=='combine_label':
      return (label,label, label), select
    image_front = tf.image.decode_jpeg(example['image_front'], channels=3)
    image_side =  tf.image.decode_jpeg(example['image_side'], channels=3)

    if with_transform:
      image_front = augment_img(image_front)
      image_side = augment_img(image_side)
    else: 
      image_front = augment_img_wot(image_front)
      image_side = augment_img_wot(image_side)
    if output_mode=='both':
      return [image_front, image_side], label, select
    elif output_mode=='img_view':
      return [image_front, image_side], example['view'], select
    elif output_mode=='both_view':
      return [image_front, image_side], label, example['view'], select
    else:
      return [image_front, image_side], select

In [ ]:
option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

### Data loader

In [ ]:
def select_image(*record):
  select = record[-1]
  return select == 1

def remove_select(*record):
  return record[:-1]

In [ ]:
def select_view_cat(view, *record):
  current_view = record[-1]
  return current_view == view

def remove_view(*record):
  return record[:-1]

In [ ]:
def select_view(view, *record):
  record = record[0]
  img = None
  if view == 'frontal':
    img = record[0][0]
  elif view == 'side':
    img = record[0][1]
  else: 
    img = record[0]
  if len(record) == 1:
    return img
  return img, record[1]

In [ ]:
def preprocess_image(image):
  image = tf.cast(image, tf.float32)
  image = image / 255.
  image_net_mean = np.array([0.485, 0.456, 0.406])
  image_net_std = np.array([0.229, 0.224, 0.225])
  image = tf.math.subtract(image, image_net_mean)
  image = tf.math.divide(image, image_net_std)
  return image
def preprocess_image_dataset(image, label=None, replicate_label=False):
  image = preprocess_image(image)
  if len(image.shape) ==5:
    image = (image[:,0], image[:,1])
  if label is None:
    return image
  if replicate_label:
    label = (label, label, label)
  return image, label

In [ ]:
def choose_view(ds, view):
  ds = ds.filter(lambda *record: select_view_cat(view, *record)).map(remove_view, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  return ds

In [ ]:
def choose_selected(ds):
  ds = ds.filter(select_image).map(remove_select, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  return ds

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/*.tfrec')
train_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
train_dsrl = train_dsr.shuffle(300000).with_options(option_no_order)
train_dsrl = train_dsrl.map(lambda record : read_tfrecord(record, 'both', True), num_parallel_calls=tf.data.experimental.AUTOTUNE)
#  select only images that were randomly selected
train_dsrs = choose_selected(train_dsrl)

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_val_mul_nr/*.tfrec')
val_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
val_dsrl = val_dsr.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)

#  select only images that were randomly selected
val_dsrs = choose_selected(val_dsrl)

In [ ]:
train_dsrl = train_dsr.shuffle(300000).with_options(option_no_order)

train_dsrs_view = train_dsrl.map(lambda record : read_tfrecord(record, 'both_view'), num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dsrs_view = val_dsr.map(lambda record : read_tfrecord(record, 'both_view'), num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dsrs_view = choose_selected(train_dsrs_view)
val_dsrs_view = choose_selected(val_dsrs_view)

train_dsrs_view = choose_view(train_dsrs_view, SELECTED_VIEW).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
val_dsrs_view = choose_view(val_dsrs_view, SELECTED_VIEW).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_test_mul_nr/*.tfrec')
test_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
test_dsrl = test_dsr.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_dsrs_img = test_dsr.map(lambda record : read_tfrecord(record, 'img'), num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dsrs_lab = test_dsr.map(lambda record : read_tfrecord(record, 'label'), num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dsrs_lab3 = test_dsr.map(lambda record : read_tfrecord(record, 'combine_label'), num_parallel_calls=tf.data.experimental.AUTOTUNE)

#  select only images that were randomly selected
test_dsrs = choose_selected(test_dsrl)
test_dsrs_img = choose_selected(test_dsrs_img)
test_dsrs_lab = choose_selected(test_dsrs_lab)
test_dsrs_lab3 = choose_selected(test_dsrs_lab3)

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_test_mul_nr/*.tfrec')
test_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)

test_dsrs_img_view = test_dsr.map(lambda record : read_tfrecord(record, 'both_view'), num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dsrs_lab_view = test_dsr.map(lambda record : read_tfrecord(record, 'view_label'), num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_dsrs_img_view = choose_selected(test_dsrs_img_view)
test_dsrs_lab_view = choose_selected(test_dsrs_lab_view)

test_dsrs_img_view = choose_view(test_dsrs_img_view, SELECTED_VIEW).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
test_dsrs_lab_view = choose_view(test_dsrs_lab_view, SELECTED_VIEW).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
combine_train_ds = train_dsrs.map(lambda *record : select_view('both', record)).batch(BS)

# two side images, one label
combine_train_dsr = combine_train_ds.map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
# two side images, three label
combine_full_train_dsr = combine_train_ds.map(lambda img, lab : preprocess_image_dataset(img, lab, True), num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
frontal_train_dsr = train_dsrs.map(lambda *record : select_view('frontal',record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_train_dsr = train_dsrs.map(lambda *record : select_view( 'side',record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
combine_val_ds = val_dsrs.map(lambda *record : select_view('both', record)).batch(BS)

combine_val_dsr = combine_val_ds.map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
combine_full_val_dsr = combine_val_ds.map(lambda img, lab : preprocess_image_dataset(img, lab, True), num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
frontal_val_dsr = val_dsrs.map(lambda *record : select_view('frontal',record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_val_dsr = val_dsrs.map(lambda *record : select_view( 'side', record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
combine_test_dsr_img = test_dsrs.map(lambda *record : select_view( 'both', record)).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
frontal_test_dsr_img = test_dsrs.map(lambda *record : select_view( 'frontal',  record)).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_test_dsr_img = test_dsrs.map(lambda *record : select_view('side', record)).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

test_dsr_lab3 = test_dsrs_lab3.prefetch(tf.data.experimental.AUTOTUNE)
test_dsr_lab = test_dsrs_lab.prefetch(tf.data.experimental.AUTOTUNE)
# labels for specific view
frontal_test_dsr_lab = test_dsrs.map(lambda *record : select_view( 'frontal', record)).map(lambda img, lab: lab, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_test_dsr_lab  = test_dsrs.map(lambda *record : select_view('side',record)).map(lambda img, lab: lab, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

### Data counter

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/*.tfrec')
dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE).with_options(option_no_order)
dsr = dsr.map(lambda record : read_tfrecord(record, "view_label"), num_parallel_calls=tf.data.experimental.AUTOTUNE)

#  select only images that were randomly selected
dsrs = dsr.filter(select_image).map(remove_select, num_parallel_calls=tf.data.experimental.AUTOTUNE)

ds_set = list(dsrs)

In [ ]:
ap_ll = []
ap_la = []
pa_ll = []
pa_la = []

In [ ]:
for lab, view  in ds_set:
  view = view.numpy().decode('UTF-8')
  if view == 'ap_ll':
    ap_ll.append(lab.numpy())
  elif view =='ap_la':
    ap_la.append(lab.numpy())
  elif view == 'pa_ll':
    pa_ll.append(lab.numpy())
  elif view == 'pa_la':
    pa_la.append(lab.numpy())

In [ ]:
len(ap_la)

12560

In [ ]:
len(ap_ll)

1647

In [ ]:
len(pa_la)

39838

In [ ]:
len(pa_ll)

19486

In [ ]:
dataset = pa_la

In [ ]:
dataset = np.array(tf.stack(dataset))
dataset = dataset[:,0,:]

In [ ]:
counts = []
for i in range(14):
  tmp = dataset[:,i]
  counts.append(np.count_nonzero(tmp == 1))

In [ ]:
counts

[3036, 2640, 533, 1140, 277, 674, 971, 4852, 24677, 2513, 257, 2043, 266, 956]

## **`Gradient Blending`**

In [ ]:
def initialize_loss_cache(model, training_ds, val_ds):
  models = [model] + list(separate_model(model))
  loss_cache = np.zeros([4, 2])
  for i in range(len(models)):
    current_model = models[i]
    val_loss = compute_loss(current_model, val_ds[i], 5)
    train_loss = compute_loss(current_model,training_ds[i], 5)
    loss_cache[i] = [train_loss, val_loss]
  return loss_cache

In [ ]:
def compute_overfit(val_loss, train_loss, prev_val_loss, prev_train_loss):
  new_O = val_loss - train_loss
  prev_O = prev_val_loss - prev_train_loss
  return new_O - prev_O

def compute_gen(val_loss, prev_val_loss):
  return val_loss - prev_val_loss

In [ ]:
def compute_loss(model, data, wk):
  scores = model.evaluate(data, use_multiprocessing= True, workers = wk, verbose=1)
  return scores[0]

In [ ]:
def compute_coef(model, train_set, val_loss, prev_losses, wk):
  train_loss = compute_loss(model, train_set, wk)
  overfit = compute_overfit(val_loss, train_loss, prev_losses[1], prev_losses[0])
  gen = compute_gen(val_loss, prev_losses[1])
  return gen / (overfit * overfit)

In [ ]:
def compute_loss_coef(models, training_set, val_set, loss_cache, ep, wk=5):
  num_models = len(models)
  new_model_dirs = []
  coef = np.zeros(num_models - 1)
  for i in range(1, num_models):
    if i == 1:
      val_loss = 0.179598
      model = tf.keras.models.load_model('/content/drive/My Drive/OGR+/DenseNet/model_frontal/frontalmodel-010-0.172871-0.179598.h5')
      coef[i - 1] = compute_coef(model, training_set[i], val_loss, loss_cache[i], wk)
      models[i] = '/content/drive/My Drive/OGR+/DenseNet/model_frontal/frontalmodel-010-0.172871-0.179598.h5'
      continue

    if i == 2:
      val_loss = 0.177716
      model = tf.keras.models.load_model('/content/drive/My Drive/OGR+/DenseNet/model_combined/combinedmodel-008-0.168526-0.177716.h5')
      coef[i - 1] = compute_coef(model, training_set[i], val_loss, loss_cache[i], wk)
      models[i] = '/content/drive/My Drive/OGR+/DenseNet/model_combined/combinedmodel-008-0.168526-0.177716.h5'
      continue

    model = models[i]
    model = tf.keras.models.load_model(model)
    callbacks = set_callback(MODEL_LOG_DIR.format(type = MODEL_TYPES[i]), MODEL_CP_DIR.format(type = MODEL_TYPES[i]))
    history = model.fit(
        training_set[i],
        epochs=ep, 
        callbacks=callbacks,
        validation_data=val_set[i],
        use_multiprocessing=True,
        workers=wk
    )

    val_loss = np.amin(history.history['val_loss'])

    model_address = sorted(glob.glob( MODEL_CP_DIR.format(type = MODEL_TYPES[i]) + "*.h5"), key=os.path.getmtime)[-1]
    model = tf.keras.models.load_model(model_address)
    
    models[i] = model_address
    coef[i - 1] = compute_coef(model, training_set[i], val_loss, loss_cache[i], wk)

  coef_sum = np.sum(coef)
  return coef / coef_sum

In [ ]:
def offline_gradient_blending(models, training_set, val_set, coef, loss_cache, ep, **kwargs):
  losses = {
	  "front_output": "binary_crossentropy",
    "combined_output": "binary_crossentropy",
	  "side_output": "binary_crossentropy"
  }
  loss_weights = {"front_output": coef[0], "combined_output": coef[1], "side_output": coef[2]}
  main_model = models[0]
  main_model = tf.keras.models.load_model(main_model)
  optimizer = Adam(learning_rate=INIT_LR)
  main_model.compile(optimizer=optimizer, loss=losses, loss_weights=loss_weights,
	  metrics=[tf.keras.metrics.AUC(multi_label=True)])
  
  if kwargs['test_img']:
    callbacks = set_callback(MODEL_LOG_DIR.format(type = MODEL_TYPES[0]), MODEL_CP_DIR.format(type = MODEL_TYPES[0]), MAIN_AUC_FILE_DIR,kwargs['test_img'],kwargs['test_lab'])
  else:
    callbacks = set_callback(MODEL_LOG_DIR.format(type = MODEL_TYPES[0]), MODEL_CP_DIR.format(type = MODEL_TYPES[0]))
  history = main_model.fit(
      training_set[0],
      epochs=ep, 
      callbacks=callbacks,
      validation_data=val_set[0],
      use_multiprocessing=True,
      workers=5
  )

## **Model creation**

### Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, GlobalAveragePooling2D, concatenate, Lambda, Reshape
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121

def load_base_model(input, name):
  base_model = DenseNet121(weights="imagenet", include_top=False,
	input_tensor=input)
  for layer in base_model.layers:
    layer._name = layer.name + str(name)
  return base_model

In [ ]:
def create_main_model():
  input_front = Input(shape=(224,224,3), name = 'front_input')
  input_side = Input(shape=(224,224,3), name = 'side_input')

  base_model_front = load_base_model(input_front, "_front")
  base_model_side = load_base_model(input_side, "_side")

  headModel1 = base_model_front.output
  headModel2 = base_model_side.output

  headModel1 = GlobalAveragePooling2D(name='front_conv')(headModel1)
  headModel2 = GlobalAveragePooling2D(name='side_conv')(headModel2)
  headModelCombine = concatenate([headModel1, headModel2], axis=-1)
  headModelCombine = Dense(14, activation="sigmoid", name='combined_output')(headModelCombine)
  headModel1 = Dense(14, activation="sigmoid", name='front_output')(headModel1)
  headModel2 = Dense(14, activation="sigmoid", name='side_output')(headModel2)

  output = [headModel1, headModelCombine, headModel2]
  input = [input_front, input_side]

  model = Model(inputs=input, outputs=output)

  optimizer = Adam(learning_rate=INIT_LR)
  model.compile(loss="binary_crossentropy", optimizer=optimizer,
	  metrics=[tf.keras.metrics.AUC(multi_label=True)])
  return model

In [ ]:
def separate_model(model):
  inputs = model.inputs
  outputs = model.outputs
  model_front = Model(inputs=inputs[0], outputs=outputs[0])
  model_combine = Model(inputs=inputs, outputs=outputs[1])
  model_side = Model(inputs=inputs[1], outputs=outputs[2])

  optimizer = Adam(learning_rate=INIT_LR)
  model_front.compile(loss="binary_crossentropy", optimizer=optimizer,
	  metrics=[tf.keras.metrics.AUC(multi_label=True)])
  model_combine.compile(loss="binary_crossentropy", optimizer=optimizer,
	  metrics=[tf.keras.metrics.AUC(multi_label=True)])
  model_side.compile(loss="binary_crossentropy", optimizer=optimizer,
	  metrics=[tf.keras.metrics.AUC(multi_label=True)])

  return  model_front, model_combine, model_side

In [ ]:
def initialize_model_cp(model, loss_cache):
  child_models = separate_model(model)
  checkpoint_address = MODEL_CP_DIR + 'model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5'
  model_dirs = []
  for i in range(len(child_models) + 1):
    dest_dir = checkpoint_address.format(type=MODEL_TYPES[i], epoch=0, loss=loss_cache[i][0], val_loss=loss_cache[i][1])
    if i == 0:
      model.save(dest_dir)
    else:
      child_models[i - 1].save(dest_dir)
    model_dirs.append(dest_dir)
  return model_dirs

In [ ]:
main_model = create_main_model()

29089792/29084464 [==============================] - 0s 0us/step


### Callbacks

In [ ]:
from sklearn.metrics import roc_auc_score
import csv

class AUCCallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir, img, lab):
      self.img_set = img
      self.lab_set = lab
      self.log_dir = log_dir
        
    def on_epoch_end(self, epoch, logs={}):
      pred = self.model.predict(self.img_set, verbose=1)[1]
      rocs = np.zeros(14)
      for j in range(14):
        rocs[j] = roc_auc_score(self.lab_set[:,j], pred[:,j])
        
      with open(self.log_dir, 'a') as fp:
        writer = csv.writer(fp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(rocs)
      return

In [ ]:
def create_auc_logger(logdir, auc_test_img, auc_test_lab):
  test_label = list(auc_test_lab)
  test_label = np.array(tf.stack(test_label))
  test_label = test_label[:,0,:]
  return AUCCallback(logdir, auc_test_img,test_label)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler

def set_callback(tblogdir, checkpoint_address=None, auc_log_dir=None, auc_test_img=None, auc_test_lab=None):
  tensorboard_callback = TensorBoard(tblogdir, histogram_freq=1)
 
  callbacks = [
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-6),
             tensorboard_callback
  ]

  if checkpoint_address:
    checkpoint_address += 'model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5'
    callbacks += [
            EarlyStopping(patience=3, verbose=1),
            ModelCheckpoint(checkpoint_address, save_best_only=True, monitor='val_loss')
    ]

  if auc_log_dir:
    auc_logger = create_auc_logger(auc_log_dir,auc_test_img,auc_test_lab)
    callbacks.append(auc_logger)
  return callbacks

## **Training Process**

In [ ]:
training_set = [combine_full_train_dsr, frontal_train_dsr, combine_train_dsr, side_train_dsr]

val_set = [combine_full_val_dsr, frontal_val_dsr, combine_val_dsr, side_val_dsr] 

test_img = [combine_test_dsr_img, frontal_test_dsr_img, combine_test_dsr_img, side_test_dsr_img]
test_lab = [test_dsr_lab3, frontal_test_dsr_lab, test_dsr_lab, side_test_dsr_lab]    

### Baseline Training

In [ ]:
_, combine_model, _ = separate_model(main_model)

In [ ]:
tblogdir = os.path.join(BASELINE_MODEL_LOG_DIR, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(tblogdir, histogram_freq=1)

In [ ]:
auc_callback = create_auc_logger(BASELINE_MODEL_AUC_LOG_DIR, test_dsrs_img_view, test_dsrs_lab_view)

In [ ]:
callbacks = [
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-6),
            #  EarlyStopping(patience=3, verbose=1),
            #  ModelCheckpoint(BASELINE_MODEL_CP_DIR, save_best_only=True, monitor='val_loss'),
             tensorboard_callback,
            #  auc_callback
  ]

In [ ]:
callbacks = set_callback(MODEL_LOG_DIR.format(type = MODEL_TYPES[i]))

In [ ]:
history = combine_model.fit(
        combine_train_dsr,
        epochs=10, 
        validation_data= combine_val_dsr,
        callbacks=callbacks,
        use_multiprocessing=True,
        workers=5
    )

In [ ]:
model = tf.keras.models.load_model("/content/drive/My Drive/OGR+/DenseNet1/model_full/fullmodel-008-0.177664-0.181804.h5")

In [ ]:
_, combine_model, _ = separate_model(model)

In [ ]:
test_label = list(test_dsrs_lab_view)
test_label = np.array(tf.stack(test_label))
test_label = test_label[:,0,:]

In [ ]:
pred = combine_model.predict(test_dsrs_img_view, verbose=1)
rocs = np.zeros(14)
for j in range(14):
  rocs[j] = roc_auc_score(test_label[:,j], pred[:,j])

9/9 [==============================] - 33s 4s/step


In [ ]:
with open('/content/drive/My Drive/test.csv', 'a') as fp:
  writer = csv.writer(fp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  writer.writerow(rocs)

### Gradient Blending Initilization

In [ ]:
loss_cache = initialize_loss_cache(main_model, training_set, val_set)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2298/2298 [==============================] - 320s 139ms/step - loss: 0.9453 - auc_3: 0.5130


In [ ]:
loss_cache

array([[2.2797482 , 2.23741102],
       [0.78479326, 0.75781667],
       [0.54984999, 0.51677203],
       [0.94528401, 0.96282196]])

In [ ]:
loss_cache = np.array([[2.2797482 , 2.23741102],
       [0.78479326, 0.75781667],
       [0.54984999, 0.51677203],
       [0.94528401, 0.96282196]])

In [ ]:
model_dirs = initialize_model_cp(main_model, loss_cache)

In [ ]:
model_dirs

['/content/drive/My Drive/OGR+/DenseNet/model_full/fullmodel-000-2.279748-2.237411.h5',
 '/content/drive/My Drive/OGR+/DenseNet/model_frontal/frontalmodel-000-0.784793-0.757817.h5',
 '/content/drive/My Drive/OGR+/DenseNet/model_combined/combinedmodel-000-0.549850-0.516772.h5',
 '/content/drive/My Drive/OGR+/DenseNet/model_side/sidemodel-000-0.945284-0.962822.h5']

In [ ]:
model_dirs = ['/content/drive/My Drive/OGR+/DenseNet/model_full/fullmodel-000-2.279748-2.237411.h5',
 '/content/drive/My Drive/OGR+/DenseNet/model_frontal/frontalmodel-000-0.784793-0.757817.h5',
 '/content/drive/My Drive/OGR+/DenseNet/model_combined/combinedmodel-000-0.549850-0.516772.h5',
 '/content/drive/My Drive/OGR+/DenseNet/model_side/sidemodel-000-0.945284-0.962822.h5']

### Offline Gradient Blending

In [ ]:
coef = compute_loss_coef(model_dirs, training_set, val_set, loss_cache, 10)

2298/2298 [==============================] - 389s 169ms/step - loss: 0.1662 - auc_5: 0.8673
Epoch 1/10
      2/Unknown - 2s 1s/step - loss: 0.7816 - auc_6: 0.4491  WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.092099). Check your callbacks.
2298/2298 [==============================] - 505s 220ms/step - loss: 0.2092 - auc_6: 0.7487 - val_loss: 0.1966 - val_auc_6: 0.7860 - lr: 1.0000e-04
Epoch 2/10
2298/2298 [==============================] - 503s 219ms/step - loss: 0.1952 - auc_6: 0.7872 - val_loss: 0.1950 - val_auc_6: 0.7937 - lr: 1.0000e-04
Epoch 3/10
2298/2298 [==============================] - 503s 219ms/step - loss: 0.1919 - auc_6: 0.7989 - val_loss: 0.1905 - val_auc_6: 0.7991 - lr: 1.0000e-04
Epoch 4/10
2298/2298 [==============================] - 503s 219ms/step - loss: 0.1892 - auc_6: 0.8078 - val_loss: 0.1935 - val_auc_6: 0.8000 - lr: 1.0000e-04
Epoch 5/10
2298/2298 [==============================] - 506s 220ms/step - loss: 0.1820 - auc_

In [ ]:
coef

array([0.02370877, 0.00913429, 0.96715694])

In [ ]:
offline_gradient_blending(model_dirs, training_set, val_set, coef, loss_cache, EPOCH, test_img=combine_test_dsr_img, test_lab= test_dsr_lab)

Epoch 1/10
      2/Unknown - 3s 1s/step - loss: 0.7755 - front_output_loss: 0.7539 - combined_output_loss: 0.5950 - side_output_loss: 0.7777 - front_output_auc_7: 0.3499 - combined_output_auc_7: 0.4030 - side_output_auc_7: 0.5045  WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.162618). Check your callbacks.
2298/2298 [==============================] - 1052s 458ms/step - loss: 0.2091 - front_output_loss: 0.2063 - combined_output_loss: 0.2027 - side_output_loss: 0.2092 - front_output_auc_7: 0.7614 - combined_output_auc_7: 0.7613 - side_output_auc_7: 0.7494 - val_loss: 0.1946 - val_front_output_loss: 0.1911 - val_combined_output_loss: 0.1883 - val_side_output_loss: 0.1947 - val_front_output_auc_7: 0.8033 - val_combined_output_auc_7: 0.8026 - val_side_output_auc_7: 0.7860 - lr: 1.0000e-04
Epoch 2/10
2298/2298 [==============================] - 1035s 450ms/step - loss: 0.1948 - front_output_loss: 0.1912 - combined_output_loss: 0.1875 - side_output_los